## 📗 Prompt template Basics 📗 ##

In [22]:
from typing import Dict, List

from langchain_community.llms import Ollama
llm = Ollama(model="llama3",temperature=0.6)

### 📄 Prompt templates using from_template method 📄

In [23]:
from langchain_core.prompts import PromptTemplate

# 1st way of creating a prompt template. 
# using "from_template" method of PromptTemplate class.
pt = PromptTemplate.from_template('tell me a funny joke about {content}')     # --> returns an object of PromptTemplate class.


# we can format it just like string formatting.
print(pt.format(content = 'elephant'))                  # --> prints prompt as string.


# we can also invoke a pt object as it is runnable.
final_prompt = pt.invoke({"content":"elephant"})        
final_prompt                                            # --> returns StringPromptValue instance
print(final_prompt)                                     # --> prints a string variable with prompt as its value.                  

tell me a funny joke about elephant
text='tell me a funny joke about elephant'


### 🛠️ prompt templates using constructors 🛠️

In [25]:
# using PromptTemplate class, using PromptTemplate class constructor directly.

pt = PromptTemplate(
    template='Tell me a {adjective} joke about {content}',
    input_variables=['adjective', 'content']
)

print(pt.format(adjective='good',content= 'elephant'))             # --> prints prompt as string.

final_prompt = pt.invoke({"adjective" : "funny", "content" : "elephant"})
final_prompt                                                        # --> returns StringPromptValue instance
print(final_prompt)                                                 # --> prints a string variable with prompt as its value.


Tell me a good joke about elephant
text='Tell me a funny joke about elephant'


### 🦙 Send prompt to LLM 🦙

In [26]:
# > we can send the prompt to llm by two ways. <#
# >   1. Directly invoking the llm
# >   2. Creating a chain of two objects.


# 1st is directly invoking the llm object.
# llm.invoke(final_prompt)                            # --> returns a non parsed string output.


# 2nd way we can chain prompt and llm object. chain direction is left --> right.
chain = pt | llm
chain.invoke(
    {"adjective": "new", "content": "elephant"}
)  # --> returns a non parsed string output.

# Second way with chaining is preferred !

"Here's one:\n\nWhy did the elephant quit his job at the circus?\n\nBecause he was tired of working for peanuts and wanted to trumpet his own success!\n\nHope that made you trumpet with laughter!"

### 📖 few shot example prompt templates 📖

In [31]:
from langchain_core.prompts import FewShotPromptTemplate

# first create a list of dictionaries as examples.
user_provided_examples: List[Dict[str, str]] = [
    {
        "question": "what is 2 * 2 ?",
        "answer": "4",
    },
    {
        "question": "what is 5 * 3 ?",
        "answer": "8",
    },
    {
        "question": "what is 12 * 2 ?",
        "answer": "14",
    },
]


# then we will create a template showing how to present examples to the LLM.
example_template = """
    Question : {question}
    Answer : {answer}
"""


# we will use that template to create a prompt_template
example_pt = PromptTemplate(
    template=example_template,
    input_variables=["question", "answer"],
)

# Checking the format of our examples prompt template.
example_pt.invoke(user_provided_examples[0]).to_string()


system_message = ''' A User is trying to teach you something new by giving examples below in the form of question and answers. For every question there is an answer provided, try to relate the answer to its respective question. learn the pattern through the question and answer pair given as example. While learning this pattern keep an open mind and try to answer the final question asked by the user based on this new pattern. Examples are - '''



# Then we'll create a few shots prompt template using our examples and example_prompt_template
fewshot_pt = FewShotPromptTemplate(
    examples = user_provided_examples,
    example_prompt = example_pt,

    prefix = system_message,                # We can also pass a system message to using prefix.
    suffix="Question: {question}",          # suffix is a prompt_template for final question, which is not an example.
    input_variables=["question"],
)


# Finally we'll invoke the fewshot_pt to create final prompt that will be sent to the llm.
final_prompt = fewshot_pt.invoke({"question": "In the given examples please explain what does c stands for and what is 10 c 2 ?"})
print(final_prompt)

text=' A User is trying to teach you something new by giving examples below in the form of question and answers. For every question there is an answer provided, try to relate the answer to its respective question. learn the pattern through the question and answer pair given as example. While learning this pattern keep an open mind and try to answer the final question asked by the user based on this new pattern. Examples are - \n\n\n    Question : what is 2 * 2 ?\n    Answer : 4\n\n\n\n    Question : what is 5 * 3 ?\n    Answer : 8\n\n\n\n    Question : what is 12 * 2 ?\n    Answer : 14\n\n\nQuestion: In the given examples please explain what does c stands for and what is 10 c 2 ?'


### ⛓️‍💥 Chaining all the parts together ⛓️‍💥

In [32]:
from langchain_core.output_parsers import StrOutputParser

# use String parser to get output in string 
str_parser = StrOutputParser()


chain = fewshot_pt | llm | str_parser
# chain.invoke({"question": "what is 10 c 2 ?"})

### 🦙 Stream the FewShotExample prompt result from LLM 🦙

In [34]:
line = ''

# If we want to stream output line by line.
for chunk in chain.stream({"question": "Based on the examples what do you think '*' means and what is 12 * 12 ?"}):
    if chunk == '.' or chunk == '\n':
        line += chunk
        print(line)
        line = ""
    
    else:
        line += chunk

print(line)

Based on the examples, I notice that the answers seem to be increasing by a fixed amount each time.
 Specifically:

* 2 * 2 = 4 (increase of 2)
* 5 * 3 = 8 (increase of 3)
* 12 * 2 = 14 (increase of 2)

This pattern suggests that '*' might mean "add" or "increment by", rather than the typical meaning of multiplication.

If this is correct, then:

* 12 * 12 would be an increase of 12 to 12, which would result in an answer of... 24!

Am I on the right track?


### 🪢 Ollama Embeddings 🪢

In [5]:
from langchain_community.embeddings import OllamaEmbeddings

embedder = OllamaEmbeddings(model="llama3")

embeddings  = embedder.embed_query('Hi My name is Sahil')

print(len(embeddings))
print(embeddings[:5])


4096
[-2.3860483169555664, 3.483680486679077, -3.3421359062194824, -1.697510004043579, 1.383670687675476]


### 📖Semantic similarity example selection 📖

In [7]:

#? from langchain_core.prompts import FewShotPromptTemplate

from typing import List, Dict
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import OllamaEmbeddings

# Create an embedder object.
embedder = OllamaEmbeddings(model="llama3")


# # then we will create a template showing how to present examples to the LLM.
example_template : str = """
    Question : {question}
    Answer : {answer}
"""

# first create an extensive list of dictionaries as examples.
user_provided_examples: List[Dict[str, str]] = [
    {
        "question": "what is 2 c 2 ?",
        "answer": "4",
    },
    {
        "question": "what is 5 c 3 ?",
        "answer": "15",
    },
    {
        "question": "what is 12 c 2 ?",
        "answer": "24",
    },
    {
        "question": "what is 5 m 2 ?",
        "answer": "2.5",
    },
    {
        "question": "what is 12 m 3 ?",
        "answer": "4",
    },
    {
        "question": "what is 15 m 15 ?",
        "answer": "1",
    },
]


# Create an example selector 
example_selector = SemanticSimilarityExampleSelector.from_examples(
    user_provided_examples,                 # This is the list of examples available to select from.
    embedder,                               # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    Chroma,                                 # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    k=4,                                    # This is the number of examples to produce.
)



# Select the most similar example to the input.
question = "what is 10 m 2 ?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")





# # we will use that template to create a prompt_template
# example_pt = PromptTemplate(
#     template=example_template,
#     input_variables=["question", "answer"],
# )

# # Checking the format of our examples prompt template.
# example_pt.invoke(user_provided_examples[0]).to_string()


# system_message = ''' A Human is trying to teach you something new by giving examples below in the form of question and answers. For every question there is an answer provided that is suitable for the human. Try to learn from these examples and answer the final question asked by the user. Examples are - '''



# # Then we'll create a few shots prompt template using our examples and example_prompt_template
# fewshot_pt = FewShotPromptTemplate(
#     examples = user_provided_examples,
#     example_prompt = example_pt,

#     prefix = system_message,                # We can also pass a system message to using prefix.
#     suffix="Question: {question}",          # suffix is a prompt_template for final question, which is not an example.
#     input_variables=["question"],
# )


# # Finally we'll invoke the fewshot_pt to create final prompt that will be sent to the llm.
# final_prompt = fewshot_pt.invoke({"question": "what is 10 c 2 ?"})
# print(final_prompt)

Examples most similar to the input: what is 10 m 2 ?


answer: 2.5
question: what is 5 m 2 ?


answer: 2.5
question: what is 5 m 2 ?


answer: 2.5
question: what is 5 m 2 ?


answer: 4
question: what is 12 m 3 ?
